In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install fastf1
import os
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

project_path = "/content/drive/MyDrive/F1-Strategy-Lab"
cache_path = project_path + "/fastf1_cache"
data_path = project_path + "/data"

os.makedirs(cache_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

fastf1.Cache.enable_cache(cache_path)

print("Drive mounted.")
print("Project folder:", project_path)
print("Cache folder:", cache_path)
print("Data export folder:", data_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ModuleNotFoundError: No module named 'fastf1'

In [ ]:
session = fastf1.get_session(2024, 'Bahrain', 'R')
session.load()

print("Session loaded:", session.name, session.date)


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.7.0]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_d

Session loaded: Race 2024-03-02 15:00:00


In [ ]:
laps = session.laps
laps.head()


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:01:37.489000,VER,1,0 days 00:01:37.284000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:41.266000,...,False,Red Bull Racing,0 days 00:59:59.911000,2024-03-02 15:03:42.342,12,1.0,False,,False,False
1,0 days 01:03:13.785000,VER,1,0 days 00:01:36.296000,2.0,1.0,NaT,NaT,0 days 00:00:30.916000,0 days 00:00:41.661000,...,False,Red Bull Racing,0 days 01:01:37.489000,2024-03-02 15:05:19.920,1,1.0,False,,False,True
2,0 days 01:04:50.538000,VER,1,0 days 00:01:36.753000,3.0,1.0,NaT,NaT,0 days 00:00:30.999000,0 days 00:00:41.966000,...,False,Red Bull Racing,0 days 01:03:13.785000,2024-03-02 15:06:56.216,1,1.0,False,,False,True
3,0 days 01:06:27.185000,VER,1,0 days 00:01:36.647000,4.0,1.0,NaT,NaT,0 days 00:00:30.931000,0 days 00:00:41.892000,...,False,Red Bull Racing,0 days 01:04:50.538000,2024-03-02 15:08:32.969,1,1.0,False,,False,True
4,0 days 01:08:04.358000,VER,1,0 days 00:01:37.173000,5.0,1.0,NaT,NaT,0 days 00:00:31.255000,0 days 00:00:42.056000,...,False,Red Bull Racing,0 days 01:06:27.185000,2024-03-02 15:10:09.616,1,1.0,False,,False,True


In [ ]:
# Save laps data
laps.to_csv(f"{data_path}/laps_2024_bahrain.csv", index=False)

# Save telemetry for a selected driver (example: VER)
ver = laps.pick_driver('VER').get_car_data().add_distance()
ver.to_csv(f"{data_path}/telemetry_VER_2024_bahrain.csv", index=False)

print("Files saved to Google Drive:")
print(f"- {data_path}/laps_2024_bahrain.csv")
print(f"- {data_path}/telemetry_VER_2024_bahrain.csv")


/usr/local/lib/python3.12/dist-packages/fastf1/core.py:3081: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


Files saved to Google Drive:
- /content/drive/MyDrive/F1-Strategy-Lab/data/laps_2024_bahrain.csv
- /content/drive/MyDrive/F1-Strategy-Lab/data/telemetry_VER_2024_bahrain.csv


Loop to download many races (2021–2024)

In [ ]:
!pip install fastf1
import os
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import fastf1
import pandas as pd

# Define paths and enable cache, as the initial setup cell failed to execute
project_path = "/content/drive/MyDrive/F1-Strategy-Lab"
cache_path = os.path.join(project_path, "fastf1_cache")
data_path = os.path.join(project_path, "data")

os.makedirs(cache_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

fastf1.Cache.enable_cache(cache_path)

# Seasons you want to download
seasons = [2024]

def slugify_event_name(name: str) -> str:
    """Turn 'Bahrain Grand Prix' → 'bahrain_grand_prix'."""
    return (
        name.lower()
        .replace(" ", "_")
        .replace("-", "_")
        .replace("'", "")
    )

for year in seasons:
    print(f"\n===== Season {year} =====")
    schedule = fastf1.get_event_schedule(year)

    # Loop over all events that have a race ("R")
    for _, row in schedule.iterrows():
        event_name = row["EventName"]

        safe_event = slugify_event_name(event_name)
        event_folder = os.path.join(data_path, str(year), safe_event)
        os.makedirs(event_folder, exist_ok=True)

        out_file = os.path.join(event_folder, f"laps_{year}_{safe_event}.csv")

        # Skip if already downloaded
        if os.path.exists(out_file):
            print(f"[SKIP] {year} {event_name} (already exists)")
            continue

        try:
            print(f"[LOAD] {year} {event_name} – race session")
            session = fastf1.get_session(year, event_name, "R")
            session.load()

            laps = session.laps
            laps.to_csv(out_file, index=False)

            print(f"[OK] Saved laps → {out_file}")
        except Exception as e:
            print(f"[ERROR] {year} {event_name}: {e}")


===== Season 2024 =====


events      WARNING 	Correcting user input 'Pre-Season Testing' to 'Singapore Grand Prix'
core           INFO 	Loading data for Singapore Grand Prix - Race [v3.7.0]
INFO:fastf1.fastf1.core:Loading data for Singapore Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...


[LOAD] 2024 Pre-Season Testing – race session


req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
DEBUG:fastf1.ergast:Failed to parse timestamp '-1:55:59.335' in Ergastresponse.
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
INFO:fastf1.api:Fetching session status data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
INFO:fastf1.fastf

[OK] Saved laps → /content/drive/MyDrive/F1-Strategy-Lab/data/2024/pre_season_testing/laps_2024_pre_season_testing.csv
[LOAD] 2024 Bahrain Grand Prix – race session


...

[OK] Saved laps → /content/drive/MyDrive/F1-Strategy-Lab/data/2024/las_vegas_grand_prix/laps_2024_las_vegas_grand_prix.csv
[LOAD] 2024 Qatar Grand Prix – race session


req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
INFO:fastf1.api:Fetching session status data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
INFO:fastf1.fastf1.req:No cached data found for lap_count. Loading data...
_api           INFO 	F

[OK] Saved laps → /content/drive/MyDrive/F1-Strategy-Lab/data/2024/qatar_grand_prix/laps_2024_qatar_grand_prix.csv
[LOAD] 2024 Abu Dhabi Grand Prix – race session


req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
DEBUG:fastf1.ergast:Failed to parse timestamp '-1:58:03.658' in Ergastresponse.
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
INFO:fastf1.api:Fetching session status data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
INFO:fastf1.fastf

[OK] Saved laps → /content/drive/MyDrive/F1-Strategy-Lab/data/2024/abu_dhabi_grand_prix/laps_2024_abu_dhabi_grand_prix.csv
